In [1]:
#preprocessing
import os
import pandas as pd
from pathlib import Path

input_dir = 'C:/Users/it shop/Downloads/archive'
output_dir = 'E:/Project DEPI/output preprocessing'

Path(output_dir).mkdir(parents=True, exist_ok=True)

def process_ecg(data):
    print("Processing ECG data...")
    data = data.dropna()
    if 'ECG_Signal' in data.columns:
        data['ECG_Signal'] = (data['ECG_Signal'] - data['ECG_Signal'].min()) / (data['ECG_Signal'].max() - data['ECG_Signal'].min())
    return data

def process_eeg(data):
    print("Processing EEG data...")
    data = data.dropna()
    if 'EEG_Signal' in data.columns:
        data['EEG_Signal'] = (data['EEG_Signal'] - data['EEG_Signal'].mean()) / data['EEG_Signal'].std()
    return data

ratio_of_alpha_beta_file_path = 'C:/Users/it shop/Downloads/archive/Ratio of Alpha _ Beta Power.xlsx'
ratio_alpha_data = pd.read_excel(ratio_of_alpha_beta_file_path, sheet_name='Alpha')
ratio_beta1_data = pd.read_excel(ratio_of_alpha_beta_file_path, sheet_name='Beta1')
ratio_beta2_data = pd.read_excel(ratio_of_alpha_beta_file_path, sheet_name='Beta2')

def drop_columns(df):
    subject_col_idx = df.columns.get_loc('Subject (No.)')
    gender_col_idx = df.columns.get_loc('Gender')
    cols_to_keep = list(df.columns[:gender_col_idx + 1]) + list(df.columns[gender_col_idx + 13:])
    return df[cols_to_keep]

def preprocess_and_reshape(df, sheet_name):
    df = df.drop([0, 1]).reset_index(drop=True)
    df = drop_columns(df)
    
    df.columns = ['Subject NO.', 'Gender', 'EO_Fp 1 - Fp 2', 'EO_F 3 - F 4', 'EO_T 3 - T 4', 'EO_P 3 - P 4',
                  'AC1_Fp 1 - Fp 2', 'AC1_F 3 - F 4', 'AC1_T 3 - T 4', 'AC1_P 3 - P 4',
                  'AC2_Fp 1 - Fp 2', 'AC2_F 3 - F 4', 'AC2_T 3 - T 4', 'AC2_P 3 - P 4']
    
    reshaped_data = pd.DataFrame()
    periods = ['EO', 'AC1', 'AC2']
    for period in periods:
        block = df[['Subject NO.', 'Gender',
                    f'{period}_Fp 1 - Fp 2', f'{period}_F 3 - F 4', f'{period}_T 3 - T 4', f'{period}_P 3 - P 4']].copy()
        
        block.columns = ['Subject NO.', 'Gender', f'{sheet_name}_(Fp 1 - Fp 2)', f'{sheet_name}_(F 3 - F 4)',
                         f'{sheet_name}_(T 3 - T 4)', f'{sheet_name}_(P 3 - P 4)']
        
        block['Segment'] = period
        reshaped_data = pd.concat([reshaped_data, block], ignore_index=True)
    
    return reshaped_data

alpha_data = preprocess_and_reshape(ratio_alpha_data, 'Alpha')
beta1_data = preprocess_and_reshape(ratio_beta1_data, 'Beta1')
beta2_data = preprocess_and_reshape(ratio_beta2_data, 'Beta2')

final_df = alpha_data.merge(beta1_data, on=['Subject NO.', 'Gender', 'Segment'])
final_df = final_df.merge(beta2_data, on=['Subject NO.', 'Gender', 'Segment'])

cols = final_df.columns.tolist()
cols.insert(2, cols.pop(cols.index('Segment')))  
final_df = final_df[cols]


def preprocess_file(file_path, output_dir):
    _, file_extension = os.path.splitext(file_path)
    
    if file_extension == '.csv':
        data = pd.read_csv(file_path)
    elif file_extension in ['.xls', '.xlsx']:
        data = pd.read_excel(file_path)  
    else:
        print(f"Unsupported file type: {file_path}. Skipping...")
        return
    
    
    if 'ECG' in file_path:
        data = process_ecg(data)
    elif 'EEG' in file_path:
        data = process_eeg(data)
    elif 'Ratio' in file_path:
        print(f"Processing Ratio data from {file_path}...")
    else:
        print(f"File '{file_path}' does not match any known data type (ECG/EEG/Ratio). Skipping...")
        return
    
    # Save the processed data to the output directory
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    data.to_excel(output_file, index=False, engine='openpyxl')  
    print(f"Processed file saved to: {output_file}")


for file_name in os.listdir(input_dir):
    file_path = os.path.join(input_dir, file_name)
    preprocess_file(file_path, output_dir)


final_output_file = os.path.join(output_dir, 'final_output.csv')  
final_df.to_csv(final_output_file, index=False)  
print(f"Final merged data saved to: {final_output_file}")


Processing ECG data...
Processed file saved to: E:/Project DEPI/output preprocessing\ECG (EO, AC1, AC2).xlsx
Processing EEG data...
Processed file saved to: E:/Project DEPI/output preprocessing\EEG (EO, AC1, AC2).xlsx
Processing Ratio data from C:/Users/it shop/Downloads/archive\Ratio of Alpha _ Beta Power.xlsx...
Processed file saved to: E:/Project DEPI/output preprocessing\Ratio of Alpha _ Beta Power.xlsx
Final merged data saved to: E:/Project DEPI/output preprocessing\final_output.csv


In [2]:
#sql script
import os
import pyodbc

connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-I5L38OD;"  
    "DATABASE=stressFeatures;"
    "Trusted_Connection=yes;"
)


db_conn = pyodbc.connect(connection_string)
cursor = db_conn.cursor()

sql_dir = 'E:/Project DEPI/SQL query'

sql_files = sorted([f for f in os.listdir(sql_dir) if f.endswith('.sql')])

for sql_file in sql_files:
    with open(os.path.join(sql_dir, sql_file), 'r') as f:
        sql_script = f.read()
        

    commands = sql_script.split('GO')
    
    for command in commands:
        command = command.strip()  
        if command:  
            cursor.execute(command)
            print(f"Executed: {command}")

db_conn.commit()
db_conn.close()

print("SQL scripts executed successfully.")


Executed: use stressFeatures;
Executed: IF OBJECT_ID ( 'create_ecg_storedProcedure', 'P' ) IS NOT NULL
    DROP PROCEDURE create_ecg_storedProcedure;
Executed: CREATE PROCEDURE create_ecg_storedProcedure
AS
BEGIN

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'EO' 
and TABLE_NAME = 'ECG(Males)')
BEGIN
    CREATE TABLE EO.[ECG(Males)] (
    SubjectNO INT PRIMARY KEY,
    "Mean HR (BPM)" FLOAT,
    "AVNN (ms)" FLOAT,
    "SDNN (ms)" FLOAT,
    "NN50 (beats)" INT,
    "pNN50 (%)" FLOAT,
    "RMSSD (ms)" FLOAT,
    "LF (ms2)" FLOAT,
    "LF Norm (n.u.)" FLOAT,
    "HF (ms2)" FLOAT,
    "HF Norm (n.u.)" FLOAT,
    "LF/HF Ratio" FLOAT,

    FOREIGN KEY (SubjectNO) REFERENCES Members(SubjectNO)
    )
END

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'EO' 
and TABLE_NAME = 'ECG(Females)')
BEGIN
    SELECT *
    INTO EO.[ECG(Females)]
    FROM EO.[ECG(Males)]
    WHERE 1 = 0;
END 

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLE

In [3]:
#inserting
import os
import pyodbc

connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-I5L38OD;" 
    "DATABASE=stressFeatures;"  
    "Trusted_Connection=yes;"
)

try:
    db_conn = pyodbc.connect(connection_string)
    cursor = db_conn.cursor()
    print("Connected to the database successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit()

sql_dir = 'E:/Project DEPI/insert'  

for sql_file in os.listdir(sql_dir):
    if sql_file.endswith('.sql'):
        with open(os.path.join(sql_dir, sql_file), 'r') as f:
            sql_script = f.read()
            commands = sql_script.split('GO')
            for command in commands:
                command = command.strip()  
                if command:  
                    try:
                        cursor.execute(command)
                        print(f"Executed command from script: {sql_file}")  
                    except Exception as e:
                        print(f"Error executing command from {sql_file}: {e}")


try:
    db_conn.commit()
    print("All changes committed successfully.")
except Exception as e:
    print(f"Error committing changes: {e}")


try:
    db_conn.close()
    print("Database connection closed.")
except Exception as e:
    print(f"Error closing database connection: {e}")

print("All SQL scripts executed successfully.")




Connected to the database successfully.
All changes committed successfully.
Database connection closed.
All SQL scripts executed successfully.


In [4]:
#ML
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.constraints import max_norm
from keras.regularizers import l2

df1 = pd.read_csv('E:/Project DEPI/combined_data.csv')

combined_data = pd.concat([df1], ignore_index=True)


def build_and_train_model(gender_data, gender):
    print(f"\nTraining model for {gender}...\n")

    required_columns = ['Segment', 'Subject NO.', 'Gender']
    if not all(col in gender_data.columns for col in required_columns):
        print(f"Missing required columns in {gender} data.")
        return

    X = gender_data.drop(columns=required_columns)
    y_binary = gender_data['Segment'].apply(lambda x: 0 if x == 'EO' else 1)

    # Normalization
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binary, test_size=0.3, random_state=42)

    # Reshape for LSTM
    X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Build the model
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_constraint=max_norm(3)), input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False, kernel_constraint=max_norm(3)))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.0003, clipvalue=1.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    model.fit(X_train_lstm, y_train, epochs=150, batch_size=16, verbose=1, validation_data=(X_test_lstm, y_test),
              callbacks=[reduce_lr, early_stopping])

    binary_predictions = model.predict(X_test_lstm)
    binary_predictions = (binary_predictions > 0.5).astype(int)

    binary_acc = accuracy_score(y_test, binary_predictions)
    binary_f1 = f1_score(y_test, binary_predictions)

    print(f"{gender} - EO vs AC Accuracy: {binary_acc}, F1 Score: {binary_f1}")

    model_filename = f'E:/Project DEPI/model_{gender}.h5'
    joblib.dump(model, model_filename)
    print(f"{gender} model trained and saved as {model_filename}.")

male_data = combined_data[combined_data['Gender'] == 'Male']
female_data = combined_data[combined_data['Gender'] == 'Female']

build_and_train_model(male_data, 'Male')
build_and_train_model(female_data, 'Female')




Training model for Male...



C:\Users\it shop\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 772ms/step - accuracy: 0.6771 - loss: 0.8546 - val_accuracy: 0.5000 - val_loss: 0.8619 - learning_rate: 3.0000e-04
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5343 - loss: 0.8593 - val_accuracy: 0.5000 - val_loss: 0.8609 - learning_rate: 3.0000e-04
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7831 - loss: 0.8413 - val_accuracy: 0.5000 - val_loss: 0.8603 - learning_rate: 3.0000e-04
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8221 - loss: 0.8404 - val_accuracy: 0.5000 - val_loss: 0.8596 - learning_rate: 3.0000e-04
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.7290 - loss: 0.8419 - val_accuracy: 0.5000 - val_loss: 0.8589 - learning_rate: 3.0000e-04
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.7468 - loss: 0.8264 - val_accuracy: 0.5000 - val_loss: 0.8583 - learning_rate: 3.0000e-04
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7390 - los

C:\Users\it shop\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 797ms/step - accuracy: 0.3033 - loss: 0.8656 - val_accuracy: 0.6316 - val_loss: 0.8614 - learning_rate: 3.0000e-04
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4226 - loss: 0.8623 - val_accuracy: 0.5263 - val_loss: 0.8593 - learning_rate: 3.0000e-04
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4155 - loss: 0.8623 - val_accuracy: 0.5263 - val_loss: 0.8572 - learning_rate: 3.0000e-04
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6662 - loss: 0.8477 - val_accuracy: 0.6842 - val_loss: 0.8552 - learning_rate: 3.0000e-04
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6690 - loss: 0.8542 - val_accuracy: 0.6316 - val_loss: 0.8534 - learning_rate: 3.0000e-04
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5646 - loss: 0.8533 - val_accuracy: 0.5789 - val_loss: 0.8516 - learning_rate: 3.0000e-04
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6193 - los

In [ ]:
# in vc write code streamlit , save (app.py) , open folder app and open cmd to run . when run in cmd write (streamlit run app.py)